In [1]:
%matplotlib inline


Word2Vec Model
==============

Introduces Gensim's Word2Vec model and demonstrates its use on the `Lee Evaluation Corpus
<https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf>`_.



In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In case you missed the buzz, Word2Vec is a widely used algorithm based on neural
networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow).
Using large amounts of unannotated plain text, word2vec learns relationships
between words automatically. The output are vectors, one vector per word,
with remarkable linear relationships that allow us to do things like:

* vec("king") - vec("man") + vec("woman") =~ vec("queen")
* vec("Montreal Canadiens") – vec("Montreal") + vec("Toronto") =~ vec("Toronto Maple Leafs").

Word2vec is very useful in `automatic text tagging
<https://github.com/RaRe-Technologies/movie-plots-by-genre>`_\ , recommender
systems and machine translation.

This tutorial:

#. Introduces ``Word2Vec`` as an improvement over traditional bag-of-words
#. Shows off a demo of ``Word2Vec`` using a pre-trained model
#. Demonstrates training a new model from your own data
#. Demonstrates loading and saving models
#. Introduces several training parameters and demonstrates their effect
#. Discusses memory requirements
#. Visualizes Word2Vec embeddings by applying dimensionality reduction

Review: Bag-of-words
--------------------

.. Note:: Feel free to skip these review sections if you're already familiar with the models.

You may be familiar with the `bag-of-words model
<https://en.wikipedia.org/wiki/Bag-of-words_model>`_ from the
`core_concepts_vector` section.
This model transforms each document to a fixed-length vector of integers.
For example, given the sentences:

- ``John likes to watch movies. Mary likes movies too.``
- ``John also likes to watch football games. Mary hates football.``

The model outputs the vectors:

- ``[1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]``
- ``[1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]``

Each vector has 10 elements, where each element counts the number of times a
particular word occurred in the document.
The order of elements is arbitrary.
In the example above, the order of the elements corresponds to the words:
``["John", "likes", "to", "watch", "movies", "Mary", "too", "also", "football", "games", "hates"]``.

Bag-of-words models are surprisingly effective, but have several weaknesses.

First, they lose all information about word order: "John likes Mary" and
"Mary likes John" correspond to identical vectors. There is a solution: bag
of `n-grams <https://en.wikipedia.org/wiki/N-gram>`__
models consider word phrases of length n to represent documents as
fixed-length vectors to capture local word order but suffer from data
sparsity and high dimensionality.

Second, the model does not attempt to learn the meaning of the underlying
words, and as a consequence, the distance between vectors doesn't always
reflect the difference in meaning.  The ``Word2Vec`` model addresses this
second problem.

Introducing: the ``Word2Vec`` Model
-----------------------------------

``Word2Vec`` is a more recent model that embeds words in a lower-dimensional
vector space using a shallow neural network. The result is a set of
word-vectors where vectors close together in vector space have similar
meanings based on context, and word-vectors distant to each other have
differing meanings. For example, ``strong`` and ``powerful`` would be close
together and ``strong`` and ``Paris`` would be relatively far.

The are two versions of this model and :py:class:`~gensim.models.word2vec.Word2Vec`
class implements them both:

1. Skip-grams (SG)
2. Continuous-bag-of-words (CBOW)

.. Important::
  Don't let the implementation details below scare you.
  They're advanced material: if it's too much, then move on to the next section.

The `Word2Vec Skip-gram <http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model>`__
model, for example, takes in pairs (word1, word2) generated by moving a
window across text data, and trains a 1-hidden-layer neural network based on
the synthetic task of given an input word, giving us a predicted probability
distribution of nearby words to the input. A virtual `one-hot
<https://en.wikipedia.org/wiki/One-hot>`__ encoding of words
goes through a 'projection layer' to the hidden layer; these projection
weights are later interpreted as the word embeddings. So if the hidden layer
has 300 neurons, this network will give us 300-dimensional word embeddings.

Continuous-bag-of-words Word2vec is very similar to the skip-gram model. It
is also a 1-hidden-layer neural network. The synthetic training task now uses
the average of multiple input context words, rather than a single word as in
skip-gram, to predict the center word. Again, the projection weights that
turn one-hot words into averageable vectors, of the same width as the hidden
layer, are interpreted as the word embeddings.




Word2Vec Demo
-------------

To see what ``Word2Vec`` can do, let's download a pre-trained model and play
around with it. We will fetch the Word2Vec model trained on part of the
Google News dataset, covering approximately 3 million words and phrases. Such
a model can take hours to train, but since it's already available,
downloading and loading it with Gensim takes minutes.

.. Important::
  The model is approximately 2GB, so you'll need a decent network connection
  to proceed.  Otherwise, skip ahead to the "Training Your Own Model" section
  below.

You may also check out an `online word2vec demo
<http://radimrehurek.com/2014/02/word2vec-tutorial/#app>`_ where you can try
this vector algebra for yourself. That demo runs ``word2vec`` on the
**entire** Google News dataset, of **about 100 billion words**.




In [3]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

2022-05-09 16:09:28,986 : INFO : Creating /Users/alexanderng/gensim-data


[=====---------------------------------------------] 10.2% 170.3/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[======--------------------------------------------] 12.8% 212.7/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=================================================-] 98.0% 1629.8/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 1662.8/1662.8MB downloaded


2022-05-09 16:11:55,587 : INFO : word2vec-google-news-300 downloaded
2022-05-09 16:11:55,590 : INFO : loading projection weights from /Users/alexanderng/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2022-05-09 16:12:41,549 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/alexanderng/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-05-09T16:12:41.547637', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


A common operation is to retrieve the vocabulary of a model. That is trivial:



In [4]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


We can easily obtain vectors for terms the model is familiar with:




In [5]:
vec_king = wv['king']

Unfortunately, the model is unable to infer vectors for unfamiliar words.
This is one limitation of Word2Vec: if this limitation matters to you, check
out the FastText model.




In [6]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


Moving on, ``Word2Vec`` supports several word similarity tasks out of the
box.  You can see how the similarity intuitively decreases as the words get
less and less similar.




In [7]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


Print the 5 most similar words to "car" or "minivan"



In [8]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

[('SUV', 0.8532191514968872), ('vehicle', 0.8175783753395081), ('pickup_truck', 0.7763689160346985), ('Jeep', 0.7567334175109863), ('Ford_Explorer', 0.7565719485282898)]


Which of the below does not belong in the sequence?



In [9]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

car


Training Your Own Model
-----------------------

To start, you'll need some data for training the model. For the following
examples, we'll use the `Lee Evaluation Corpus
<https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf>`_
(which you `already have
<https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor>`_
if you've installed Gensim).

This corpus is small enough to fit entirely in memory, but we'll implement a
memory-friendly iterator that reads it line-by-line to demonstrate how you
would handle a larger corpus.




In [10]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

2022-05-09 16:33:01,430 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-09 16:33:01,431 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2022-05-09 16:33:01,432 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2022-05-09T16:33:01.432789', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'created'}


If we wanted to do any custom preprocessing, e.g. decode a non-standard
encoding, lowercase, remove numbers, extract named entities... All of this can
be done inside the ``MyCorpus`` iterator and ``word2vec`` doesn’t need to
know. All that is required is that the input yields one sentence (list of
utf8 words) after another.

Let's go ahead and train a model on our corpus.  Don't worry about the
training parameters much for now, we'll revisit them later.




In [11]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

2022-05-09 16:33:20,086 : INFO : collecting all words and their counts
2022-05-09 16:33:20,089 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:33:20,166 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-09 16:33:20,167 : INFO : Creating a fresh vocabulary
2022-05-09 16:33:20,176 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1750 unique words (25.068041827818366%% of original 6981, drops 5231)', 'datetime': '2022-05-09T16:33:20.176019', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:33:20,176 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 49335 word corpus (84.83801073049938%% of original 58152, drops 8817)', 'datetime': '2022-05-09T16:33:20.176803', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 

Once we have our model, we can use it in the same way as in the demo above.

The main part of the model is ``model.wv``\ , where "wv" stands for "word vectors".




In [12]:
vec_king = model.wv['king']

Retrieving the vocabulary works the same way:



In [13]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


Storing and loading models
--------------------------

You'll notice that training non-trivial models can take time.  Once you've
trained your model and it works as expected, you can save it to disk.  That
way, you don't have to spend time training it all over again later.

You can store/load models using the standard gensim methods:




In [14]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

2022-05-09 16:33:37,749 : INFO : Word2Vec lifecycle event {'fname_or_handle': '/var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-05-09T16:33:37.749111', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'saving'}
2022-05-09 16:33:37,750 : INFO : not storing attribute cum_table
2022-05-09 16:33:37,755 : INFO : saved /var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf
2022-05-09 16:33:37,756 : INFO : loading Word2Vec object from /var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf
2022-05-09 16:33:37,758 : INFO : loading wv recursively from /var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf.wv.* with mmap=None
2022-05-09 16:33:37,759 : INFO : setting ignored attribute cum_table to None
2022-05-09 16:33:37,774

which uses pickle internally, optionally ``mmap``\ ‘ing the model’s internal
large NumPy matrices into virtual memory directly from disk files, for
inter-process memory sharing.

In addition, you can load models created by the original C tool, both using
its text and binary formats::

  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)




Training Parameters
-------------------

``Word2Vec`` accepts several parameters that affect both training speed and quality.

min_count
---------

``min_count`` is for pruning the internal dictionary. Words that appear only
once or twice in a billion-word corpus are probably uninteresting typos and
garbage. In addition, there’s not enough data to make any meaningful training
on those words, so it’s best to ignore them:

default value of min_count=5



In [15]:
model = gensim.models.Word2Vec(sentences, min_count=10)

2022-05-09 16:33:39,378 : INFO : collecting all words and their counts
2022-05-09 16:33:39,380 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:33:39,451 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-09 16:33:39,452 : INFO : Creating a fresh vocabulary
2022-05-09 16:33:39,456 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 889 unique words (12.734565248531728%% of original 6981, drops 6092)', 'datetime': '2022-05-09T16:33:39.456725', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:33:39,457 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 43776 word corpus (75.2785802723896%% of original 58152, drops 14376)', 'datetime': '2022-05-09T16:33:39.457413', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26

vector_size
-----------

``vector_size`` is the number of dimensions (N) of the N-dimensional space that
gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more
accurate) models. Reasonable values are in the tens to hundreds.




In [16]:
# The default value of vector_size is 100.
model = gensim.models.Word2Vec(sentences, vector_size=200)

2022-05-09 16:33:41,682 : INFO : collecting all words and their counts
2022-05-09 16:33:41,684 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:33:41,757 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-09 16:33:41,758 : INFO : Creating a fresh vocabulary
2022-05-09 16:33:41,766 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1750 unique words (25.068041827818366%% of original 6981, drops 5231)', 'datetime': '2022-05-09T16:33:41.766048', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:33:41,766 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 49335 word corpus (84.83801073049938%% of original 58152, drops 8817)', 'datetime': '2022-05-09T16:33:41.766920', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 

workers
-------

``workers`` , the last of the major parameters (full list `here
<http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec>`_)
is for training parallelization, to speed up training:




In [17]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2022-05-09 16:33:43,738 : INFO : collecting all words and their counts
2022-05-09 16:33:43,739 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:33:43,813 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-09 16:33:43,814 : INFO : Creating a fresh vocabulary
2022-05-09 16:33:43,821 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1750 unique words (25.068041827818366%% of original 6981, drops 5231)', 'datetime': '2022-05-09T16:33:43.821591', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:33:43,822 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 49335 word corpus (84.83801073049938%% of original 58152, drops 8817)', 'datetime': '2022-05-09T16:33:43.822326', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 

The ``workers`` parameter only has an effect if you have `Cython
<http://cython.org/>`_ installed. Without Cython, you’ll only be able to use
one core because of the `GIL
<https://wiki.python.org/moin/GlobalInterpreterLock>`_ (and ``word2vec``
training will be `miserably slow
<http://rare-technologies.com/word2vec-in-python-part-two-optimizing/>`_\ ).




Memory
------

At its core, ``word2vec`` model parameters are stored as matrices (NumPy
arrays). Each array is **#vocabulary** (controlled by the ``min_count`` parameter)
times **vector size** (the ``vector_size`` parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number
to two, or even one). So if your input contains 100,000 unique words, and you
asked for layer ``vector_size=200``\ , the model will require approx.
``100,000*200*4*3 bytes = ~229MB``.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would
take a few megabytes), but unless your words are extremely loooong strings, memory
footprint will be dominated by the three matrices above.




Evaluating
----------

``Word2Vec`` training is an unsupervised task, there’s no good way to
objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic
test examples, following the “A is to B as C is to D” task. It is provided in
the 'datasets' folder.

For example a syntactic analogy of comparative type is ``bad:worse;good:?``.
There are total of 9 types of syntactic comparisons in the dataset like
plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as
capital cities (``Paris:France;Tokyo:?``) or family members
(``brother:sister;dad:?``).




Gensim supports the same evaluation set, in exactly the same format:




In [18]:
model.wv.evaluate_word_analogies(datapath('questions-words.txt'))

2022-05-09 16:33:48,218 : INFO : Evaluating word analogies for top 300000 words in the model on /Volumes/GDRIVE_SSD/Applications/anaconda3/anaconda3/envs/sps620env/lib/python3.7/site-packages/gensim/test/test_data/questions-words.txt
2022-05-09 16:33:48,229 : INFO : capital-common-countries: 0.0% (0/6)
2022-05-09 16:33:48,241 : INFO : capital-world: 0.0% (0/2)
2022-05-09 16:33:48,252 : INFO : family: 0.0% (0/6)
2022-05-09 16:33:48,262 : INFO : gram3-comparative: 0.0% (0/20)
2022-05-09 16:33:48,269 : INFO : gram4-superlative: 0.0% (0/12)
2022-05-09 16:33:48,275 : INFO : gram5-present-participle: 0.0% (0/20)
2022-05-09 16:33:48,286 : INFO : gram6-nationality-adjective: 0.0% (0/30)
2022-05-09 16:33:48,294 : INFO : gram7-past-tense: 0.0% (0/20)
2022-05-09 16:33:48,305 : INFO : gram8-plural: 0.0% (0/30)
2022-05-09 16:33:48,307 : INFO : Quadruplets with out-of-vocabulary words: 99.3%
2022-05-09 16:33:48,308 : INFO : NB: analogies containing OOV words were skipped from evaluation! To change t

(0.0,
 [{'section': 'capital-common-countries',
   'correct': [],
   'incorrect': [('CANBERRA', 'AUSTRALIA', 'KABUL', 'AFGHANISTAN'),
    ('CANBERRA', 'AUSTRALIA', 'PARIS', 'FRANCE'),
    ('KABUL', 'AFGHANISTAN', 'PARIS', 'FRANCE'),
    ('KABUL', 'AFGHANISTAN', 'CANBERRA', 'AUSTRALIA'),
    ('PARIS', 'FRANCE', 'CANBERRA', 'AUSTRALIA'),
    ('PARIS', 'FRANCE', 'KABUL', 'AFGHANISTAN')]},
  {'section': 'capital-world',
   'correct': [],
   'incorrect': [('CANBERRA', 'AUSTRALIA', 'KABUL', 'AFGHANISTAN'),
    ('KABUL', 'AFGHANISTAN', 'PARIS', 'FRANCE')]},
  {'section': 'currency', 'correct': [], 'incorrect': []},
  {'section': 'city-in-state', 'correct': [], 'incorrect': []},
  {'section': 'family',
   'correct': [],
   'incorrect': [('HE', 'SHE', 'HIS', 'HER'),
    ('HE', 'SHE', 'MAN', 'WOMAN'),
    ('HIS', 'HER', 'MAN', 'WOMAN'),
    ('HIS', 'HER', 'HE', 'SHE'),
    ('MAN', 'WOMAN', 'HE', 'SHE'),
    ('MAN', 'WOMAN', 'HIS', 'HER')]},
  {'section': 'gram1-adjective-to-adverb', 'correct': [

This ``evaluate_word_analogies`` method takes an `optional parameter
<http://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.evaluate_word_analogies>`_
``restrict_vocab`` which limits which test examples are to be considered.




In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset
specific to your business based on it. It contains word pairs together with
human-assigned similarity judgments. It measures the relatedness or
co-occurrence of two words. For example, 'coast' and 'shore' are very similar
as they appear in the same context. At the same time 'clothes' and 'closet'
are less similar because they are related but not interchangeable.




In [19]:
model.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

2022-05-09 16:33:53,610 : INFO : Pearson correlation coefficient against /Volumes/GDRIVE_SSD/Applications/anaconda3/anaconda3/envs/sps620env/lib/python3.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1650
2022-05-09 16:33:53,611 : INFO : Spearman rank-order correlation coefficient against /Volumes/GDRIVE_SSD/Applications/anaconda3/anaconda3/envs/sps620env/lib/python3.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1394
2022-05-09 16:33:53,611 : INFO : Pairs with unknown words ratio: 83.0%


((0.16495299845037242, 0.2078531439547672),
 SpearmanrResult(correlation=0.13944699266364144, pvalue=0.2879488080747829),
 83.0028328611898)

.. Important::
  Good performance on Google's or WS-353 test set doesn’t mean word2vec will
  work well in your application, or vice versa. It’s always best to evaluate
  directly on your intended task. For an example of how to use word2vec in a
  classifier pipeline, see this `tutorial
  <https://github.com/RaRe-Technologies/movie-plots-by-genre>`_.




Online training / Resuming training
-----------------------------------

Advanced users can load a model and continue training it with more sentences
and `new vocabulary words <online_w2v_tutorial.ipynb>`_:




In [20]:
model = gensim.models.Word2Vec.load(temporary_filepath)
more_sentences = [
    ['Advanced', 'users', 'can', 'load', 'a', 'model',
     'and', 'continue', 'training', 'it', 'with', 'more', 'sentences'],
]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.epochs)

# cleaning up temporary file
import os
os.remove(temporary_filepath)

2022-05-09 16:33:55,402 : INFO : loading Word2Vec object from /var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf
2022-05-09 16:33:55,405 : INFO : loading wv recursively from /var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf.wv.* with mmap=None
2022-05-09 16:33:55,405 : INFO : setting ignored attribute cum_table to None
2022-05-09 16:33:55,419 : INFO : Word2Vec lifecycle event {'fname': '/var/folders/1t/9bsyqzz12_b49yhmd0tnnc380000gn/T/gensim-model-6hnby_uf', 'datetime': '2022-05-09T16:33:55.419674', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'loaded'}
2022-05-09 16:33:55,420 : INFO : collecting all words and their counts
2022-05-09 16:33:55,421 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:33:55,421 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
202

You may need to tweak the ``total_words`` parameter to ``train()``,
depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C
tool, ``KeyedVectors.load_word2vec_format()``. You can still use them for
querying/similarity, but information vital for training (the vocab tree) is
missing there.




Training Loss Computation
-------------------------

The parameter ``compute_loss`` can be used to toggle computation of loss
while training the Word2Vec model. The computed loss is stored in the model
attribute ``running_training_loss`` and can be retrieved using the function
``get_latest_training_loss`` as follows :




In [21]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

2022-05-09 16:33:57,120 : INFO : collecting all words and their counts
2022-05-09 16:33:57,122 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:33:57,197 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-09 16:33:57,197 : INFO : Creating a fresh vocabulary
2022-05-09 16:33:57,219 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 6981 unique words (100.0%% of original 6981, drops 0)', 'datetime': '2022-05-09T16:33:57.219690', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:33:57,220 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 58152 word corpus (100.0%% of original 58152, drops 0)', 'datetime': '2022-05-09T16:33:57.220461', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 

1511609.375


Benchmarks
----------

Let's run some benchmarks to see effect of the training loss computation code
on training time.

We'll use the following data for the benchmarks:

#. Lee Background corpus: included in gensim's test data
#. Text8 corpus.  To demonstrate the effect of corpus size, we'll look at the
   first 1MB, 10MB, 50MB of the corpus, as well as the entire thing.




In [22]:
import io
import os

import gensim.models.word2vec
import gensim.downloader as api
import smart_open


def head(path, size):
    with smart_open.open(path) as fin:
        return io.StringIO(fin.read(size))


def generate_input_data():
    lee_path = datapath('lee_background.cor')
    ls = gensim.models.word2vec.LineSentence(lee_path)
    ls.name = '25kB'
    yield ls

    text8_path = api.load('text8').fn
    labels = ('1MB', '10MB', '50MB', '100MB')
    sizes = (1024 ** 2, 10 * 1024 ** 2, 50 * 1024 ** 2, 100 * 1024 ** 2)
    for l, s in zip(labels, sizes):
        ls = gensim.models.word2vec.LineSentence(head(text8_path, s))
        ls.name = l
        yield ls


input_data = list(generate_input_data())

[==================================================] 100.0% 31.6/31.6MB downloaded


2022-05-09 16:34:04,499 : INFO : text8 downloaded


We now compare the training time taken for different combinations of input
data and model training parameters like ``hs`` and ``sg``.

For each combination, we repeat the test several times to obtain the mean and
standard deviation of the test duration.




In [23]:
# Temporarily reduce logging verbosity
logging.root.level = logging.ERROR

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

fast = True
if fast:
    input_data_subset = input_data[:3]
else:
    input_data_subset = input_data


for data in input_data_subset:
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(
                        data,
                        compute_loss=loss_flag,
                        sg=sg_val,
                        hs=hs_val,
                        seed=seed_val,
                    )
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)

                model_result = {
                    'train_data': data.name,
                    'compute_loss': loss_flag,
                    'sg': sg_val,
                    'hs': hs_val,
                    'train_time_mean': time_mean,
                    'train_time_std': time_std,
                }
                print("Word2vec model #%i: %s" % (len(train_time_values), model_result))
                train_time_values.append(model_result)

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(
    by=['train_data', 'sg', 'hs', 'compute_loss'],
    ascending=[False, False, True, False],
)
print(train_times_table)

2022-05-09 16:34:09,215 : INFO : collecting all words and their counts
2022-05-09 16:34:09,216 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-09 16:34:09,233 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2022-05-09 16:34:09,233 : INFO : Creating a fresh vocabulary
2022-05-09 16:34:09,241 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1762 unique words (16.343567387069847%% of original 10781, drops 9019)', 'datetime': '2022-05-09T16:34:09.241559', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:34:09,242 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 46084 word corpus (76.94773751878444%% of original 59890, drops 13806)', 'datetime': '2022-05-09T16:34:09.242228', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 

Word2vec model #0: {'train_data': '25kB', 'compute_loss': True, 'sg': 0, 'hs': 0, 'train_time_mean': 0.22794230779012045, 'train_time_std': 0.012309707619817731}


2022-05-09 16:34:10,103 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-05-09 16:34:10,106 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-05-09 16:34:10,109 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-05-09 16:34:10,110 : INFO : EPOCH - 4 : training on 59890 raw words (32547 effective words) took 0.0s, 905979 effective words/s
2022-05-09 16:34:10,138 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-05-09 16:34:10,139 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-05-09 16:34:10,142 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-05-09 16:34:10,143 : INFO : EPOCH - 5 : training on 59890 raw words (32557 effective words) took 0.0s, 1034243 effective words/s
2022-05-09 16:34:10,144 : INFO : Word2Vec lifecycle event {'msg': 'training on 299450 raw words (162907 effective words) took 0.2s, 928507 effective words/s', 'datetime': '2022-05-09T16:3

Word2vec model #1: {'train_data': '25kB', 'compute_loss': False, 'sg': 0, 'hs': 0, 'train_time_mean': 0.236218531926473, 'train_time_std': 0.009841611869431117}


2022-05-09 16:34:10,811 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-05-09 16:34:10,817 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-05-09 16:34:10,822 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-05-09 16:34:10,823 : INFO : EPOCH - 2 : training on 59890 raw words (32552 effective words) took 0.0s, 700692 effective words/s
2022-05-09 16:34:10,867 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-05-09 16:34:10,872 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-05-09 16:34:10,877 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-05-09 16:34:10,877 : INFO : EPOCH - 3 : training on 59890 raw words (32603 effective words) took 0.1s, 617957 effective words/s
2022-05-09 16:34:10,919 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-05-09 16:34:10,923 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-0

...

Word2vec model #16: {'train_data': '10MB', 'compute_loss': True, 'sg': 0, 'hs': 0, 'train_time_mean': 5.012579758961995, 'train_time_std': 0.02534181687433191}


2022-05-09 16:35:06,472 : INFO : collected 73167 word types from a corpus of 1788017 raw words and 179 sentences
2022-05-09 16:35:06,473 : INFO : Creating a fresh vocabulary
2022-05-09 16:35:06,554 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 20167 unique words (27.562972378257957%% of original 73167, drops 53000)', 'datetime': '2022-05-09T16:35:06.554326', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:35:06,555 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 1703716 word corpus (95.28522379820774%% of original 1788017, drops 84301)', 'datetime': '2022-05-09T16:35:06.555028', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:35:06,653 : I

Word2vec model #17: {'train_data': '10MB', 'compute_loss': False, 'sg': 0, 'hs': 0, 'train_time_mean': 4.983124653498332, 'train_time_std': 0.010357836255407028}


2022-05-09 16:35:21,436 : INFO : collected 73167 word types from a corpus of 1788017 raw words and 179 sentences
2022-05-09 16:35:21,437 : INFO : Creating a fresh vocabulary
2022-05-09 16:35:21,522 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 20167 unique words (27.562972378257957%% of original 73167, drops 53000)', 'datetime': '2022-05-09T16:35:21.522104', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:35:21,522 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 1703716 word corpus (95.28522379820774%% of original 1788017, drops 84301)', 'datetime': '2022-05-09T16:35:21.522764', 'gensim': '4.0.1', 'python': '3.7.7 (default, Mar 26 2020, 10:32:53) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-21.3.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-09 16:35:21,622 : I

KeyboardInterrupt: 

Visualising Word Embeddings
---------------------------

The word embeddings made by the model can be visualised by reducing
dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example:

* Semantic: words like cat, dog, cow, etc. have a tendency to lie close by
* Syntactic: words like run, running or cut, cutting lie close together.

Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.

.. Important::
  The model used for the visualisation is trained on a small corpus. Thus
  some of the relations might not be so clear.




In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

Conclusion
----------

In this tutorial we learned how to train word2vec models on your custom data
and also how to evaluate it. Hope that you too will find this popular tool
useful in your Machine Learning tasks!

Links
-----

- API docs: :py:mod:`gensim.models.word2vec`
- `Original C toolkit and word2vec papers by Google <https://code.google.com/archive/p/word2vec/>`_.


